In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup


In [ ]:
0%2

0

In [ ]:
ki = []
gu = []
for i in range(776):
    if i %2 == 0:
         ki.append(i)
    else:
        gu.append(i)

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/2022/業務連携/data/スタバ_店舗マスタ.xlsx", header=None)
df_shopname = df.iloc[ki, :].reset_index(drop=True)
df_shopname = pd.DataFrame(df_shopname).rename(columns={0:"店名"})
df_address = df.iloc[gu, :].reset_index(drop=True)
df_address = pd.DataFrame(df_address).rename(columns={0:"住所"})

In [ ]:
df_union = pd.concat([df_shopname, df_address], axis=1)
df_union

,店名,住所
0,皇居外苑 和田倉噴水公園店,東京都 千代田区 皇居外苑3-1
1,竹橋パレスサイド店,東京都 千代田区 一ツ橋1-1-1 竹橋パレスサイドビルディング 1F
2,ＫＤＤＩ大手町ビル店,東京都 千代田区 大手町1-8-1 ＫＤＤＩ大手町ビル 1F
3,大手町プレイス店,東京都 千代田区 大手町2-3-1 大手町プレイス
4,大手町ビル店,東京都 千代田区 大手町1-6-1 大手町ビル
...,...,...
383,昭島モリタウン店,東京都 昭島市 田中町562-1 モリタウン
384,イオンモール東久留米店,東京都 東久留米市 南沢5-17-62 イオンモール東久留米
385,清瀬駅前店,東京都 清瀬市 元町1-8-1 三福ビル
386,福生西友店,東京都 福生市 東町5-1 福生西友


In [ ]:
df_union.to_excel('starbacks_shopmaster.xlsx')

## 店舗ごとの情報のスクレイピング

トップで取得class="store-detail__content-desc"  
住所：済  
電話番号：  class="store-detail__text-detail Sodo-text number-text"  
営業時間：  store-detail__text-detail Sodo-text number-text  
定休日：  class="store-detail__text-detail number-text"  
アクセス：  class="store-detail__text-detail"  
無線LAN：  class="store-detail__text-detail Sodo-text"  
モバイル決済：  class="payment-method__item"  altタグを取得  
サービス：class="mobile-order__text Sodo-text"  
備考：対応検討

### クローリング用のリンク生成似必要な店舗IDを取得

In [ ]:
f = open('/content/drive/MyDrive/2022/業務連携/data/スタバ店舗情報.txt', 'r')
data = f.read()
f.close()

In [ ]:
data

'<ul id="store-list"><li data-i="0" data-store_id="4088" class="searching-result__item"><div class="result__item__info"><p class="result__item__info__title">皇居外苑 和田倉噴水公園店</p> <p class="result__item__info__desc">東京都 千代田区 皇居外苑3-1</p> <!----></div> <a class="result__item__btn">詳細</a></li><li data-i="1" data-store_id="75" class="searching-result__item"><div class="result__item__info"><p class="result__item__info__title">竹橋パレスサイド店</p> <p class="result__item__info__desc">東京都 千代田区 一ツ橋1-1-1 竹橋パレスサイドビルディング 1F</p> <!----></div> <a class="result__item__btn">詳細</a></li><li data-i="2" data-store_id="115" class="searching-result__item"><div class="result__item__info"><p class="result__item__info__title">ＫＤＤＩ大手町ビル店</p> <p class="result__item__info__desc">東京都 千代田区 大手町1-8-1 ＫＤＤＩ大手町ビル 1F</p> <!----></div> <a class="result__item__btn">詳細</a></li><li data-i="3" data-store_id="1540" class="searching-result__item"><div class="result__item__info"><p class="result__item__info__title">大手町プレイス店</p> <p class="re

In [ ]:
soup = BeautifulSoup(data, 'html.parser')
shop_id = soup.find_all("li")
shop_name = soup.find_all("p", class_="result__item__info__title")
shop_address = soup.find_all("p", class_="result__item__info__desc")


In [ ]:
shop_name[0].text

'皇居外苑 和田倉噴水公園店'

In [ ]:
# data-store_idの取り出し。リンク作成用
result_id_list = []
tmp_id_list = list(soup.find_all("li"))
for tmp in tmp_id_list:
    tmp = str(tmp)
    tmp_soup = BeautifulSoup(tmp, "html.parser")
    result = tmp_soup.find("li")["data-store_id"]
    result_id_list.append(result)

In [ ]:
# result__item__info__title
result_shopname_list = []
for tmp in shop_name:
    result = tmp.text
    result_shopname_list.append(result)

In [ ]:
# result__item__info__desc
result_shopaddress_list = []
for tmp in shop_address:
    result = tmp.text
    result_shopaddress_list.append(result)

In [ ]:
# チェック
print(len(result_id_list))
print(len(result_shopname_list))
print(len(result_shopaddress_list))

388
388
388


In [ ]:
result_df = pd.DataFrame({
    "店舗id": result_id_list,
    "店舗名": result_shopname_list,
    "店舗住所": result_shopaddress_list
})
result_df

,店舗id,店舗名,店舗住所
0,4088,皇居外苑 和田倉噴水公園店,東京都 千代田区 皇居外苑3-1
1,75,竹橋パレスサイド店,東京都 千代田区 一ツ橋1-1-1 竹橋パレスサイドビルディング 1F
2,115,ＫＤＤＩ大手町ビル店,東京都 千代田区 大手町1-8-1 ＫＤＤＩ大手町ビル 1F
3,1540,大手町プレイス店,東京都 千代田区 大手町2-3-1 大手町プレイス
4,432,大手町ビル店,東京都 千代田区 大手町1-6-1 大手町ビル
...,...,...,...
383,1858,よみうりランドHANA・BIYORI店,東京都 稲城市 矢野口4015-1 よみうりランドHANA・BIYORI
384,1859,よみうりランドグッジョバ!!店,東京都 稲城市 矢野口4015-1 よみうりランド
385,1450,むさし村山新青梅街道店,東京都 武蔵村山市 榎3-2-3
386,1666,イオンモールむさし村山1階店,東京都 武蔵村山市 榎1-1-3 イオンモールむさし村山


In [ ]:
result_df.tail()

,店舗id,店舗名,店舗住所
383,1858,よみうりランドHANA・BIYORI店,東京都 稲城市 矢野口4015-1 よみうりランドHANA・BIYORI
384,1859,よみうりランドグッジョバ!!店,東京都 稲城市 矢野口4015-1 よみうりランド
385,1450,むさし村山新青梅街道店,東京都 武蔵村山市 榎3-2-3
386,1666,イオンモールむさし村山1階店,東京都 武蔵村山市 榎1-1-3 イオンモールむさし村山
387,725,イオンモールむさし村山2階店,東京都 武蔵村山市 榎1-1-3


In [ ]:
# チェック
print(len(result_id_list)
print(len(shop_name))
print(len(shop_address))

388
388


### スクレイパー

In [ ]:
shop_id_list_base = result_df["店舗id"].to_list()
len(shop_id_list_base)

388

In [ ]:
print(shop_id_list_base[0:50][49])
print(shop_id_list_base[50:100][0])
# print(shop_id_list_base[100:250][0])
shop_id_list_1 = shop_id_list_base[0:50]
shop_id_list_2 = shop_id_list_base[50:100]
shop_id_list_3 = shop_id_list_base[100:150]
shop_id_list_4 = shop_id_list_base[150:300]
shop_id_list_5 = shop_id_list_base[300:]
shop_id_list_6 = shop_id_list_base[0:50]
shop_id_list_7 = shop_id_list_base[0:50]
shop_id_list_8 = shop_id_list_base[0:50]

1313
17


In [ ]:
print(shop_id_list_5)

['1065', '1253', '1727', '2128', '369', '182', '591', '38', '4066', '1079', '1533', '648', '4002', '135', '1361', '1688', '572', '387', '1885', '2190', '1132', '964', '39', '71', '341', '200', '1644', '1325', '943', '1120', '143', '1422', '1235', '589', '1497', '1456', '1212', '1515', '1600', '1872', '447', '1647', '921', '1250', '195', '1292', '2251', '1805', '811', '1400', '745', '1239', '4072', '1375', '1175', '445', '1098', '1119', '2109', '985', '1056', '1465', '633', '1155', '1324', '898', '658', '4', '1315', '1681', '637', '580', '543', '844', '1825', '1815', '1712', '479', '1800', '89', '2196', '1391', '370', '1858', '1859', '1450', '1666', '725']


In [ ]:
shop_id_list = shop_id_list_5
address_list = []
tel_list = []
business_hours_list = []
holiday_list = []

for shop_id in shop_id_list:
    try:
        r = requests.get("https://store.starbucks.co.jp/detail-"+ shop_id +"/")
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")

        # print(soup)
        top = soup.find_all("div", class_="store-detail__content--row")
        address = soup.find("div", class_="store-detail__text-detail line-height-22").text.strip()
        tel = soup.find("div", class_="store-detail__text-detail Sodo-text number-text").text.strip()
        business_hours = soup.find_all("div", class_="store-detail__text-detail Sodo-text number-text")[1].text.strip()
        holiday = soup.find("div", class_="store-detail__text-detail number-text").text.strip()
        address_list.append(address)
        tel_list.append(tel)
        business_hours_list.append(business_hours)
        holiday_list.append(holiday)

        print(address,tel, business_hours, holiday)
    except:
        r = requests.get("https://store.starbucks.co.jp/detail-"+ shop_id +"/")
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")

        # print(soup)
        top = soup.find_all("div", class_="store-detail__content--row")
        address = soup.find("div", class_="store-detail__text-detail line-height-22").text.strip()
        address_list.append(address)
        tel = soup.find("div", class_="store-detail__text-detail Sodo-text number-text").text.strip()
        tel_list.append(tel)
        business_hours = soup.find_all("div", class_="store-detail__text-detail Sodo-text number-text")[0].text.strip()
        business_hours_list.append(business_hours)
        holiday = soup.find("div", class_="store-detail__text-detail number-text").text.strip()
        holiday_list.append(holiday)
        

        print(address,tel, business_hours, holiday)
    # 一時停止
    time.sleep(10)

171-0021　東京都 豊島区 西池袋1-11-1 ルミネ池袋 03-5928-5091 07:30～21:30 不定休
171-0021　東京都 豊島区 西池袋3-28-14 03-5960-0600 月～金：07:00～22:30                       
                                                              土日祝：08:00～22:30 不定休
171-0021　東京都 豊島区 西池袋1-11-1 ルミネ池袋 03-6384-4085 11:00～22:30 不定休
171-0021　東京都 豊島区 西池袋1-1-25 03-5830-3811 07:00～22:00 不定休
171-0021　東京都 豊島区 西池袋1-17-10 エキニア池袋 03-5951-1007 月～金：07:00～22:30                       
                                                              土日祝：08:00～22:30 不定休
171-0022　東京都 豊島区 南池袋2-16-4 ＳＫビル 03-5391-0089 月～金：07:00～22:30                       
                                                              土日祝：08:00～22:30 不定休
171-0022　東京都 豊島区 南池袋1-29-1 池袋ショッピングパーク 03-5911-5661 07:00～22:30 不定休
171-0022　東京都 豊島区 南池袋1-18-17 Ｉ＆Ｋビル 1F 03-3590-0065 07:00～22:30 不定休
171-0031　東京都 豊島区 目白1-4-1 JR東日本ホテルメッツ 目白3階 03-5924-6082 月～金：06:30～22:00                       
                                                              土日祝：07:00～22:00 不定休
171-8512　東京都 

In [ ]:
df_5 = pd.DataFrame({
    "shop_id":shop_id_list,
    "address":address_list,
    "tel":tel_list,
    "business_hours":business_hours_list,
    "holiday":holiday_list
})

In [ ]:
df_5.to_excel('df_5.xlsx')

In [ ]:
df_2.tail()

,shop_id,address,tel,business_hours,holiday
45,848,106-6108 東京都 港区 六本木6-10-1 森タワー 1F,03-5771-1117,03-5771-1117,不定休
46,142,107-0052 東京都 港区 赤坂3-21-20 赤坂ロングビーチビル 1F,03-5570-1305,03-5570-1305,不定休
47,1461,107-0052 東京都 港区 赤坂1-8-1,03-5545-3307,03-5545-3307,不定休
48,1472,107-0052 東京都 港区 赤坂9-7-2 東京ミッドタウンB0103,03-5413-6531,03-5413-6531,不定休
49,202,107-0052 東京都 港区 赤坂2-17-1 赤坂溜池タワー,03-3505-6810,03-3505-6810,土 ・日・祝


## 店舗マスタの作成

In [ ]:
master = pd.read_excel("/content/drive/MyDrive/2022/業務連携/data/shopmaster_2.xlsx", dtype=str)
# 営業時間カラムミスった
master = pd.merge(master, result_df, left_on="shop_id", right_on="店舗id", how="inner")
master = master[["no", "shop_id", "店舗名", "address", "tel"]]
master

,no,shop_id,店舗名,address,tel
0,0,4088,皇居外苑 和田倉噴水公園店,100-0002 東京都 千代田区 皇居外苑3-1,03-6273-4844
1,1,75,竹橋パレスサイド店,100-0003 東京都 千代田区 一ツ橋1-1-1 竹橋パレスサイドビルディング 1F,03-3212-3510
2,2,115,ＫＤＤＩ大手町ビル店,100-0004 東京都 千代田区 大手町1-8-1 ＫＤＤＩ大手町ビル 1F,03-3231-2065
3,3,1540,大手町プレイス店,100-0004 東京都 千代田区 大手町2-3-1 大手町プレイス,03-6281-9645
4,4,432,大手町ビル店,100-0004 東京都 千代田区 大手町1-6-1 大手町ビル,03-5288-7330
...,...,...,...,...,...
383,383,1858,よみうりランドHANA・BIYORI店,206-8725 東京都 稲城市 矢野口4015-1 よみうりランドHANA・BIYORI,044-455-6352
384,384,1859,よみうりランドグッジョバ!!店,206-8725 東京都 稲城市 矢野口4015-1 よみうりランド,044-299-8576
385,385,1450,むさし村山新青梅街道店,208-0022 東京都 武蔵村山市 榎3-2-3,042-516-2371
386,386,1666,イオンモールむさし村山1階店,208-0022 東京都 武蔵村山市 榎1-1-3 イオンモールむさし村山,042-516-2037


In [ ]:
master.to_excel('starbacks_shopmaster_20221201.xlsx')